# Preliminaries

In [21]:
%pip install -q ragstack-ai-langchain[knowledge-store] beautifulsoup4 markdownify python-dotenv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.1/45.1 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.0/974.0 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 29.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.7/314.7 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 29.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.1/127.1 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 326.8/326.8 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 28.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 431.4/431.4 kB 30.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.7/274.7 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━

# Load the Astra Documentation into Graph Store

First, we'll crawl the DataStax documentation. LangChain includes a `SiteMapLoader` but it loads all of the pages into memory simultaneously, which makes it impossible to index larger sites from small environments (such as CoLab). So, we'll scrape the sitemap ourselves and iterate over the URLs, allowing us to process documents in batches and flush them to Astra DB. 

## Scrape the URLs from the Site Maps
First, we use Beautiful Soup to parse the XML content of each sitemap and get the list of URLs.
We also add a few extra URLs for external sites that are also useful to include in the index.

In [22]:
# Use sitemaps to crawl the content
SITEMAPS = [
    "https://docs.datastax.com/en/sitemap-astra-db-vector.xml",
    "https://docs.datastax.com/en/sitemap-cql.xml",
    "https://docs.datastax.com/en/sitemap-dev-app-drivers.xml",
    "https://docs.datastax.com/en/sitemap-glossary.xml",
    "https://docs.datastax.com/en/sitemap-astra-db-serverless.xml",
]

# Additional URLs to crawl for content.
EXTRA_URLS = ["https://github.com/jbellis/jvector"]

SITE_PREFIX = "astra"

from bs4 import BeautifulSoup
import requests


def load_pages(sitemap_url):
    r = requests.get(
        sitemap_url,
        headers={
            # Astra docs only return a sitemap with a user agent set.
            "User-Agent": "Mozilla/5.0 (X11; Linux x86_64; rv:58.0) Gecko/20100101 Firefox/58.0",
        },
    )
    xml = r.text

    soup = BeautifulSoup(xml, features="xml")
    url_tags = soup.find_all("url")
    for url in url_tags:
        yield (url.find("loc").text)


# For maintenance purposes, we could check only the new articles since a given time.
URLS = [url for sitemap_url in SITEMAPS for url in load_pages(sitemap_url)] + EXTRA_URLS
len(URLS)

1368

## Load the content from each URL
Next, we create the code to load each page. This performs the following steps:

1. Parses the HTML with BeautifulSoup
2. Locates the "content" of the HTML using an appropriate selector based on the URL
3. Find the link (`<a href="...">`) tags in the content and collect the absolute URLs (for creating edges).

Adding the URLs of these references to the metadata allows the graph store to create edges between the document.

In [ ]:
from langchain_community.document_loaders import AsyncHtmlLoader
from bs4 import BeautifulSoup
from langchain_core.documents import Document
from typing import AsyncIterator, Iterable
from ragstack_knowledge_store.graph_store import CONTENT_ID
from markdownify import MarkdownConverter
from ragstack_langchain.graph_store.extractors import HtmlLinkEdgeExtractor

markdown_converter = MarkdownConverter(heading_style="ATX")
html_link_extractor = HtmlLinkEdgeExtractor()


def select_content(soup: BeautifulSoup, url: str) -> BeautifulSoup:
    if url.startswith("https://docs.datastax.com/en/"):
        return soup.select_one("article.doc")
    elif url.startswith("https://github.com"):
        return soup.select_one("article.entry-content")
    else:
        return soup


async def load_pages(urls: Iterable[str]) -> AsyncIterator[Document]:
    loader = AsyncHtmlLoader(
        urls,
        requests_per_second=4,
        # Astra docs require a user agent
        header_template={
            "User-Agent": "Mozilla/5.0 (X11; Linux x86_64; rv:58.0) Gecko/20100101 Firefox/58.0"
        },
    )
    async for html in loader.alazy_load():
        url = html.metadata["source"]

        # Use the URL as the content ID.
        html.metadata[CONTENT_ID] = url

        # Apply the selectors while loading. This reduces the size of
        # the document as early as possible for reduced memory usage.
        soup = BeautifulSoup(html.page_content, "html.parser")
        content = select_content(soup, url)

        # Extract HTML links from the content.
        html_link_extractor.extract_one(html, content)

        # Convert the content to markdown
        html.page_content = markdown_converter.convert_soup(content)

        yield html

: 

## Initialize Environment
Before we initialize the Graph Store and write the documents we need to set some environment variables.
In colab, this will prompt you for input. When running locally, this will load from `.env`.

In [5]:
import os

if os.getenv("COLAB_RELEASE_TAG"):
    # (Option 1) - Set the environment variables from getpass.
    print("In colab. Using getpass/input for environment variables.")
    import getpass
    import os

    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter OpenAI API Key: ")
    os.environ["ASTRA_DB_DATABASE_ID"] = input("Enter Astra DB Database ID: ")
    os.environ["ASTRA_DB_APPLICATION_TOKEN"] = getpass.getpass(
        "Enter Astra DB Application Token: "
    )

    keyspace = input("Enter Astra DB Keyspace (Empty for default): ")
    if keyspace:
        os.environ["ASTRA_DB_KEYSPACE"] = keyspace
    else:
        os.environ.pop("ASTRA_DB_KEYSPACE", None)
else:
    print("Not in colab. Loading '.env' (see 'env.template' for example)")
    import dotenv

    dotenv.load_dotenv()

Not in colab. Loading '.env' (see 'env.template' for example)


## Initialize Cassio and Graph Store
With the environment variables set we initialize the Cassio library for talking to Cassandra / Astra DB.
We also create the `GraphStore`.

In [6]:
SITE_PREFIX = "astra_docs"

In [7]:
answer = input("Drop Tables? [(Y)es/(N)o]")
if answer.lower() in ["y", "yes"]:
    import cassio

    cassio.init(auto=True)
    from cassio.config import check_resolve_session, check_resolve_keyspace

    session = check_resolve_session()
    keyspace = check_resolve_keyspace()
    session.execute(f"DROP TABLE IF EXISTS {keyspace}.{SITE_PREFIX}_nodes")
    session.execute(f"DROP TABLE IF EXISTS {keyspace}.{SITE_PREFIX}_edges")
else:
    # Handle no / "wrong" input
    pass

In [8]:
import cassio
from langchain_openai import OpenAIEmbeddings
from ragstack_langchain.graph_store import CassandraGraphStore

cassio.init(auto=True)
embeddings = OpenAIEmbeddings()
graph_store = CassandraGraphStore(
    embeddings, node_table=f"{SITE_PREFIX}_nodes", edge_table=f"{SITE_PREFIX}_edges"
)

## Load the Documents
Finally, we fetch pages and write them to the graph store in batches of 50.

In [9]:
not_found = 0
found = 0

docs = []
async for doc in load_pages(URLS):
    if doc.page_content.startswith("\n# Page Not Found"):
        not_found += 1
        continue

    docs.append(doc)
    found += 1

    if len(docs) >= 50:
        graph_store.add_documents(docs)
        docs.clear()

if docs:
    graph_store.add_documents(docs)
print(f"{not_found} (of {not_found + found}) URLs were not found")

Fetching pages:  10%|9         | 131/1368 [00:07<01:22, 14.99it/s]

Added 152 edges


Fetching pages:  14%|#4        | 196/1368 [00:23<04:03,  4.81it/s]

Added 114 edges


Error fetching https://docs.datastax.com/en/cql/dse/developing/indexing/sai/read-write-paths.html with attempt 1/3: Cannot connect to host docs.datastax.com:443 ssl:default [nodename nor servname provided, or not known]. Retrying...
Fetching pages:  18%|#8        | 248/1368 [00:35<04:40,  3.99it/s]

...

Added 190 edges


Fetching pages:  98%|#########7| 1340/1368 [04:14<00:05,  5.22it/s]

Added 363 edges


Fetching pages: 100%|##########| 1368/1368 [04:23<00:00,  5.19it/s]


Added 120 edges
96 (of 1368) URLs were not found


# Create and execute the RAG Chains

In [10]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

llm = ChatOpenAI(model="gpt-4o")

template = """You are a helpful technical support bot. You should provide complete answers explaining the options the user has available to address their problem. Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)


def format_docs(docs):
    formatted = "\n\n".join(
        f"From {doc.metadata['content_id']}: {doc.page_content}" for doc in docs
    )
    return formatted

We'll use the following question. This is an interesting question because the ideal answer should be concise and in-depth, based on how the vector indexing is actually implemented.

In [11]:
QUESTION = "What vector indexing algorithms does Astra use?"

In [12]:
from IPython.display import display, Markdown


# Helper method to render markdown in responses to a chain.
def run_and_render(chain, question):
    result = chain.invoke(question)
    display(Markdown(result))

## Vector-Only Retrieval

In [13]:
# Depth 0 doesn't traverses edges and is equivalent to vector similarity only.
vector_retriever = graph_store.as_retriever(search_kwargs={"depth": 0})

vector_rag_chain = (
    {"context": vector_retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [14]:
run_and_render(vector_rag_chain, QUESTION)

Astra DB uses two indexing techniques to enhance vector search performance:

### 1. **JVector**
- **Description**: Astra DB Serverless (Vector) databases use the [JVector vector search engine](https://github.com/jbellis/jvector) to construct a graph index.
- **Features**:
  - **Immediate Document Addition**: New documents are added to the graph immediately, allowing for efficient searches right away.
  - **Compression**: JVector can compress vectors using quantization to save space and improve performance.

### 2. **Storage-Attached Index (SAI)**
- **Description**: SAI is an indexing technique designed to efficiently find rows that satisfy query predicates.
- **Features**:
  - **Numeric, Text, and Vector-Based Indexes**: Supports different kinds of searches.
  - **Customizable Indexes**: Allows customization based on specific requirements, such as a particular similarity function or text transformation.
  - **Performance**: When a search is run, SAI loads a superset of all possible results from storage based on the provided predicates, then evaluates the search criteria and sorts the results by vector similarity. The top `limit` results are returned to the user.

For more details, you can refer to the [Storage-Attached Indexing (SAI) Overview](https://docs.datastax.com/en/cql/astra/developing/indexing/sai/sai-overview.html).

## Graph Traversal Retrieval

In [15]:
# Depth 1 does vector similarity and then traverses 1 level of edges.
graph_retriever = graph_store.as_retriever(search_kwargs={"depth": 1})

graph_rag_chain = (
    {"context": graph_retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [16]:
run_and_render(graph_rag_chain, QUESTION)

Astra DB Serverless uses multiple indexing techniques to speed up searches, specifically for vector data. The key vector indexing techniques employed are:

1. **JVector**: Astra DB Serverless utilizes the [JVector vector search engine](https://github.com/jbellis/jvector) to construct a graph index. JVector builds on the DiskANN design and implements a single-layer graph with non-blocking concurrency control, enabling scalable construction with the number of cores. JVector's graph is represented by an on-disk adjacency list per node, with additional data stored in-line to support two-pass searches.

2. **Storage-Attached Index (SAI)**: SAI is another indexing technique used in Astra DB for efficiently finding rows that satisfy query predicates. It supports numeric, text, and vector-based indexes and can be customized based on requirements, such as a specific similarity function or text transformation. SAI loads a superset of all possible results from storage based on the predicates provided and then evaluates the search criteria, sorting results by vector similarity.

These indexing techniques, especially JVector, are designed to handle large-scale, high-dimensional vector data efficiently, making Astra DB Serverless suitable for applications requiring fast and accurate vector similarity searches.

## MMR Graph Traversal

In [17]:
mmr_graph_retriever = graph_store.as_retriever(
    search_type="mmr_traversal",
    search_kwargs={
        "k": 4,
        "fetch_k": 10,
        "depth": 2,
        # "score_threshold": 0.2,
    },
)

mmr_graph_rag_chain = (
    {"context": mmr_graph_retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [18]:
run_and_render(mmr_graph_rag_chain, QUESTION)

Astra DB Serverless uses multiple indexing techniques to facilitate efficient vector searches. Specifically, it employs the following vector indexing algorithms:

1. **JVector**: Astra DB Serverless uses the JVector vector search engine to construct a graph index. JVector is a graph-based index that builds on the DiskANN design with composable extensions. It allows for the addition of new documents to the graph immediately, enabling efficient search operations right away. JVector can also compress vectors using quantization to save space and improve performance.

2. **Storage-Attached Index (SAI)**: SAI is another indexing technique used by Astra DB Serverless to efficiently find rows that satisfy query predicates. Astra DB provides numeric-, text-, and vector-based indexes to support different kinds of searches. With SAI, searches are conducted by loading a superset of all possible results from storage based on the predicates provided. SAI then evaluates the search criteria and sorts the results by vector similarity, returning the top `limit` results to the user.

These indexing techniques are designed to optimize vector search operations, ensuring efficient retrieval and management of high-dimensional vector data.

## Check Retrieval Results

In [19]:
# Set the question and see what documents each technique retrieves.
for i, doc in enumerate(vector_retriever.invoke(QUESTION)):
    print(f"Vector [{i}]:    {doc.metadata['content_id']}")

for i, doc in enumerate(graph_retriever.invoke(QUESTION)):
    print(f"Graph [{i}]:     {doc.metadata['content_id']}")

for i, doc in enumerate(mmr_graph_retriever.invoke(QUESTION)):
    print(f"MMR Graph [{i}]: {doc.metadata['content_id']}")

Vector [0]:    https://docs.datastax.com/en/astra-db-serverless/get-started/concepts.html
Vector [1]:    https://docs.datastax.com/en/cql/astra/getting-started/vector-search-quickstart.html
Vector [2]:    https://docs.datastax.com/en/astra-db-serverless/databases/embedding-generation.html
Vector [3]:    https://docs.datastax.com/en/astra-db-serverless/get-started/astra-db-introduction.html
Graph [0]:     https://docs.datastax.com/en/astra-db-serverless/get-started/concepts.html
Graph [1]:     https://docs.datastax.com/en/cql/astra/getting-started/vector-search-quickstart.html
Graph [2]:     https://docs.datastax.com/en/astra-db-serverless/databases/embedding-generation.html
Graph [3]:     https://docs.datastax.com/en/astra-db-serverless/get-started/astra-db-introduction.html
Graph [4]:     https://docs.datastax.com/en/astra-db-serverless/databases/database-overview.html
Graph [5]:     https://docs.datastax.com/en/astra-db-serverless/integrations/semantic-kernel.html
Graph [6]:     http

# Conclusion
With vector only we retrieved chunks from the Astra documentation explaining that it used JVector.
Since it didn't follow the link to [JVector on GitHub](https://github.com/jbellis/jvector) it didn't actually answer the question.

The graph retrieval started with the same set of chunks, but it followed the edge to the documents we loaded from GitHub.
This allowed the LLM to read in more depth how JVector is implemented, which allowed it to answer the question more clearly and with more detail.